In [1]:
import pandas as pd

In [2]:
# Extract raw github csv from JHU
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

In [3]:
# read all the csv
confirmed_df = pd.read_csv(confirmed_url)
death_df = pd.read_csv(death_url)
recovered_df = pd.read_csv(recovered_url)

In [4]:
# assign name to the df (useful later)
confirmed_df.name = 'confirmed_df'
death_df.name = 'death_df'
recovered_df.name = 'recovered_df'

In [5]:
# get all the countries available in all 3 files
def all_countries():
    total_countries_available = pd.concat([confirmed_df['Country/Region'], death_df['Country/Region'], recovered_df['Country/Region']]).unique()
    column_name = pd.concat([confirmed_df['Country/Region'], death_df['Country/Region'], recovered_df['Country/Region']]).name
    number_of_countries = len(total_countries_available)
    #  if total_countries_available in pd.unique(confirmed_df['Country/Region'])
    #  total_countries_available in pd.unique(confirmed_df['Country/Region'])
    return column_name, total_countries_available
all_countries()

('Country/Region',
 array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
        'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
        'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
        'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
        'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
        'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
        'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
        'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
        'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
        "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
        'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
        'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
        'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
        'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
   

## Cleaning data

In [ ]:
# # file and country specific cleaning test
# confirmed_uk = confirmed_df[confirmed_df['Country/Region'] == 'United Kingdom']
# confirmed_uk_numbers = confirmed_uk.iloc[:,4:]
# confirmed_uk_numbers = confirmed_uk_numbers.T
# # Since there will be more than one region in UK (columns) , so we sum it to represent the whole UK cases
# confirmed_uk_numbers = confirmed_uk_numbers.sum(axis = 'columns')
# # Cleaning to daily cases will be more meaningful
# confirmed_uk_daily_numbers = confirmed_uk_numbers.diff()
# # Get the 7-day rolling average of the daily cases as well
# confirmed_uk_daily_rolling_average = confirmed_uk_daily_numbers.rolling(window = 7).mean()

In [6]:
# file and country specific cleaning test
def cleaning_data2(df_type = confirmed_df, country = ['Malaysia'], window = 7 ):
    df_countries = df_type[df_type['Country/Region'].isin(country)]    
    df_countries_numbers = df_countries.groupby(by = 'Country/Region')[df_countries.iloc[:,4:].columns].sum()
    df_countries_numbers = df_countries_numbers.T
# Cleaning to daily cases will be more meaningful
    df_countries_daily_numbers = df_countries_numbers.diff()
# Get the 7-day rolling average of the daily cases as well
    df_countries_daily_rolling_average = df_countries_daily_numbers.rolling(window = window).mean()
    df_countries_daily_rolling_average = df_countries_daily_rolling_average.rename_axis(index = 'Country/Region', columns = None).reset_index()
    df_countries_daily_rolling_average = df_countries_daily_rolling_average.rename(columns = {'Country/Region': 'Date'})
    from datetime import datetime
    df_countries_daily_rolling_average['Date'] = pd.to_datetime(df_countries_daily_rolling_average['Date']).dt.strftime('%Y-%m-%d') # convert the index date to DateTime object with the required format
    df_countries_daily_rolling_average.set_index('Date', inplace = True)
    return [df_countries_daily_rolling_average, country, window]

In [7]:
cleaning_data2(confirmed_df, ['United Kingdom', 'Malaysia', 'US', 'Singapore'])[0]

,Malaysia,Singapore,US,United Kingdom
Date,,,,
2020-01-22,NaN,NaN,NaN,NaN
2020-01-23,NaN,NaN,NaN,NaN
2020-01-24,NaN,NaN,NaN,NaN
2020-01-25,NaN,NaN,NaN,NaN
2020-01-26,NaN,NaN,NaN,NaN
...,...,...,...,...
2021-06-06,7326.000000,24.000000,14409.857143,4695.000000
2021-06-07,7104.142857,22.714286,15798.428571,5024.000000
2021-06-08,6884.285714,21.428571,14685.142857,5434.142857


In [ ]:
# # Use a custom cleaning decorator to clean the data
# from functools import wraps

# def cleaning_decorator(function): # the function here is cleaning_data
    
#     @wraps(function) # so that when __doc__ is called, function description can be called
#     def wrapper(df_type = confirmed_df, country = 'Malaysia', window = 7):
#         df_country = df_type[df_type['Country/Region'] == country]  # df filter to particular country
#         df_country_numbers = df_country.iloc[:,4:]  # get the numbers data which only starts from the 5th column
#         df_country_numbers = df_country_numbers.T  # transpose them so that dates is in the index (first column), numbers in the second column (and third or more columns because of some countries further divided to regions)
#         df_country_numbers = df_country_numbers.sum(axis = 'columns')  # aggregate the columns numbers (useful for more than 1 column numbers)
#         df_country_daily_numbers = df_country_numbers.diff()  # get the daily numbers by subtracting today with yesterday
#         df_country_daily_rolling_average = df_country_daily_numbers.rolling(window = window).mean()  # get the x-day rolling average of numbers
#         df_country_daily_rolling_average = pd.DataFrame(df_country_daily_rolling_average, columns = ['Cases']) # make it as a dataframe, with it's column named as 'Cases'
#         from datetime import datetime
#         df_country_daily_rolling_average.index = pd.to_datetime(df_country_daily_rolling_average.index).strftime('%Y-%m-%d') # convert the index date to DateTime object with the required format
#         df_country_daily_rolling_average.index.name = 'Date'  # set the index name as 'Date'
#         print(function()) # print "cleaning completed"
#         return [df_country_daily_rolling_average, country, window]  # return the following set of results 
    
#     return wrapper

# @cleaning_decorator
# def cleaning_data(df_type = confirmed_df, country = 'Malaysia', window = 7):
#     ''' 
#     Function: Daily breakdown of cases in the country
    
#     Parameters:
    
#     df_type = confirmed/death/recovered_df
#     country = country name 
#     window = x-day rolling average
#     '''
#     return "cleaning completed"

# # print(cleaning_data.__doc__)

In [ ]:
cleaning_data(df_type = confirmed_df, country = 'United Kingdom', window = 3)[0]

In [ ]:
# # Total cases worldwide
# def total_cases_worldwide(df_type):
#     total_cases_worldwide = df_type.iloc[:,-1].sum()  # sum all the last columns (which is the cumulative cases of each country) of the df
#     if df_type is confirmed_df:
#         print(f'Total Confirmed: {total_cases_worldwide: ,}')  # print the cases into thousand separators
#     elif df_type is recovered_df:
#         print(f'Total Recovered: {total_cases_worldwide: ,}')
#     elif df_type is death_df:
#         print(f'Total Death: {total_cases_worldwide: ,}')

# total_cases_worldwide(confirmed_df)

In [ ]:
# # Total cases in a country
# def total_cases_country(df_type, country):
#     df_country = df_type[df_type['Country/Region'].isin(country)]
#     total_cases_country = df_country.iloc[:,-1].sum()
#     if df_type is confirmed_df:
#         print(f'Total Confirmed in {country}: {total_cases_country}')
#     elif df_type is recovered_df:
#         print(f'Total Recovered in {country}: {total_cases_country}')
#     elif df_type is death_df:
#         print(f'Total Death in {country}: {total_cases_country}')
#     return total_cases_country
# # total_cases_country(death_df, ['Malaysia'])

In [ ]:
# total_cases_country(confirmed_df, all_countries()[1])

In [8]:
# Get all countries cumulative cases
def total_cases_all_countries(df_type):
    total_cases_country = df_type.iloc[:,-1]  # get the last column (which is the cumulative cases of each country) of the df
    total_cases_all_countries = df_type[['Country/Region', total_cases_country.name]]  # get only the 'Country/Region' column and last column from the df
    total_cases_all_countries = total_cases_all_countries.groupby(by = 'Country/Region').sum().reset_index()  # as there can be multiple same countries in the rows, group them together and sum their cases
    total_cases_all_countries.rename(columns = {total_cases_all_countries.columns[-1]: 'Cases'}, inplace=True)  # rename the column to 'Cases' and replace the old column name
    return total_cases_all_countries

In [61]:
for i in total_cases_all_countries(confirmed_df).index:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192


## Use plotly to plot the cleaned data

In [111]:
# for graphing
import plotly.express as px
# to place where the plot renders
import plotly.io as pio

In [112]:
# view plots in browser tab
pio.renderers.default = 'browser' # for pdf -> install orca first, browser view -> easiest

In [113]:
def generate_plot(df_type, country, window):
    data = cleaning_data2(df_type = df_type, country = country, window = window)  # get the daily breakdown of cases of the country
    
    figure = px.line(data[0], y = country,
                     title = f"{data[2]}-day moving average daily new {df_type.name.split('_', 1)[0]} cases")
    figure.update_xaxes(title_text = "Timeline", dtick="M1")  # step of x-ticks is 1 month
    figure.update_yaxes(title_text = "Cases")
    figure.update_layout(hovermode="x", legend_title = "Country")  # hover in the line with x appearing below for better visuals
    figure.update_traces(hovertemplate = 'Cases: %{y}')
    #figure.update_layout(hovermode="x unified")
    # figure.update_xaxes(tickangle=45,
    #     ticktext=["End of Q1", "End of Q2", "End of Q3", "End of Q4"],
    #     tickvals=["2020-04-01", "2020-07-01", "2020-10-01",data[0].index.max()])
    return figure

In [13]:
generate_plot(confirmed_df, ['United Kingdom', 'Malaysia', 'US', 'Singapore'], 7)

## DASH

In [127]:
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html

In [128]:
# create the DASH app with current notebook codes (__name__)
import dash_bootstrap_components as dbc
# MUST add the dbc themes for dbc components to work!
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [129]:
# chrome tab title
app.title = 'Covid-19 dashboard'

In [130]:
def generate_header():
    header = html.H1(children = 'Covid-19 Explorer', style = {'text-align': 'center'})
    return header

In [131]:
def metric_options_generator():
    all_metrics = ['Confirmed cases', 'Death cases', 'Recovered cases']
    options = []
    [options.append({'label': metric, 'value': metric}) for metric in all_metrics]
# Alternatively: (for loop method)
#     for metric in all_metrics:
#         option = {'label': metric, 'value': metric}
#         options.append(option)
    return options

In [119]:
metric_options_generator()

[{'label': 'Confirmed cases', 'value': 'Confirmed cases'},
 {'label': 'Death cases', 'value': 'Death cases'},
 {'label': 'Recovered cases', 'value': 'Recovered cases'}]

In [132]:
def metric_dropdown():
    dropdown = dcc.Dropdown(id = 'metric_dropdown1', 
                            options = metric_options_generator(), 
                            value = 'Confirmed cases',
                            style = {'width': '50%', 'margin-left': 'auto', 'margin-right': 'auto'}
                           )
    return dropdown
# 'width': '40%'
# style = {'width': '50%'})

In [133]:
def table_selection():
    table_selection = dash_table.DataTable(
                    id = 'datatable1',
                    columns = [
                        {"name": i, "id": i, "deletable": False, "selectable": True}
                        for i in total_cases_all_countries(confirmed_df).columns
                    ],
                    data = total_cases_all_countries(confirmed_df).to_dict('records'), # the contents of the table
                    editable=False, # do not allow edit data
                    filter_action="native", # allow filter of data by user, else "none" to deactivate the function
                    sort_action="native", # enables data to be sorted, else "none" to deactivate the function
                    sort_mode="multi",
                    column_selectable=False, # can able to tick "multi" or "single" columns 
                    row_selectable="multi",  # can able to tick "multi" or "single" columns
                    row_deletable=False,  # "False" to not allow deletion of date 
                    selected_columns=[],
                    selected_rows=[],
                    page_action="none",  # all the data is passed to the table altogether in a single page
                    style_table={'height': '500px','width' : '300px', 'overflowY': 'auto'},  # allow scrolling vertically 
                    fill_width=False,
                    style_cell_conditional=[
                        {
                            'if': {'column_id': 'Country/Region'},
                                'textAlign': 'left'
                            }
                    ],
                    style_cell = {'whiteSpace': 'normal'},
                    style_data = {'border': 'none'}
                    )

    return table_selection
# 'overflowY': 'auto
#'width' : '350px'

In [134]:
@app.callback(
    Output('datatable1', 'style_data_conditional'),
    Input('datatable1', 'selected_rows')
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

In [135]:
@app.callback(
    Output('graph1', 'figure'),
    [Input('datatable1', 'derived_virtual_data'),
    Input('datatable1', 'derived_virtual_selected_rows')]
)
def update_graph(rows, derived_virtual_selected_rows):
    dff = pd.DataFrame(rows)
    figure = generate_plot(confirmed_df, dff['Country/Region'][derived_virtual_selected_rows].tolist(), 1)
    
    return figure

In [136]:
def graph():
    graph = dcc.Graph(id = 'graph1') #figure = generate_plot(confirmed_df, ['Malaysia', 'United Kingdom'], 1)
                               #style = {'display': 'inline-block'}
                      # className = 'six columns'
                     # )
    return graph

In [137]:
app.layout = dbc.Container([
                     generate_header(),
#                      metric_dropdown(),
    dbc.Row(
        [
            dbc.Col(html.Div(html.Label('Metrics', style = {'display': 'block','text-align': 'center'})))
        ]
    ),
    
    dbc.Row(
        [
            dbc.Col(html.Div(metric_dropdown())
                   )
        ]
    ),
                     dbc.Row(
                         [
                             dbc.Col(html.Div(table_selection()), width = 3),
                             dbc.Col(html.Div(graph()), width = 9),
                         ]
                     )
                         ],fluid=True)
                                    

In [125]:
@app.callback(
    Output(component_id='graph1',component_property='figure'), #line chart
    Input(component_id='metric_dropdown1',component_property='value') #dropdown
)
def update_graph(metric_dropdown_value):
    if metric_dropdown_value == 'Confirmed cases':
        return generate_plot(confirmed_df, ['Malaysia', 'United Kingdom'], 1)
    elif metric_dropdown_value == 'Death cases':
        return generate_plot(death_df, ['Malaysia', 'United Kingdom'], 1)
    elif metric_dropdown_value == 'Recovered cases':
        return generate_plot(recovered_df, ['Malaysia', 'United Kingdom'], 1)

In [139]:
# NOTE!! If decided to stop running, need to rerun the DASH app again from the start!!
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 23:42:00] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Country/Region'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Country/Region'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Country/Region'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\

...

127.0.0.1 - - [11/Jun/2021 23:50:10] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    retu

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    retu

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\tosi0001\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    retu

127.0.0.1 - - [11/Jun/2021 23:50:10] "POST /_dash-update-component HTTP/1.1" 500 -


In [58]:
generate_plot(confirmed_df, ['Malaysia', 'United Kingdom'], 1)

In [140]:
df = confirmed_df

In [143]:
df['Country/Region'][[1,2]].to_list()

['Albania', 'Algeria']

In [147]:
if False:
    print("ok")
elif True:
    print('Ok')

Ok
